In [4]:
import requests
import shutil
import json

import pandas as pd
import numpy as np

In [5]:
pd.set_option('display.max_rows', 500)

## 1. Fetch data

In [6]:
# fetch and save play-by-play data
seasons = [20, 22, 23]
for season in seasons:
    url = f"https://github.com/ramirobentes/NBA-in-R/releases/download/pbp-poss-rs{season}/data.csv"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.content
        file_path = f"../data/pbp_{season}.csv"
        with open(file_path, "wb") as file:
            file.write(data)
        print("Data downloaded successfully.")
    else:
        print("Failed to download data. Status code:", response.status_code)

Failed to download data from season 15. Status code: 404
Failed to download data from season 16. Status code: 404
Data from season 17 downloaded successfully.
Failed to download data from season 18. Status code: 404
Failed to download data from season 19. Status code: 404
Data from season 20 downloaded successfully.
Failed to download data from season 21. Status code: 404
Data from season 22 downloaded successfully.
Data from season 23 downloaded successfully.


In [7]:
# fetch and save betting data
url = 'https://www.rotowire.com/betting/nba/tables/games-archive.php'
response = requests.get(url)
content = response.content
content = content.decode('utf-8')
betting_dict = json.loads(content)
betting_df = pd.DataFrame(betting_dict)
betting_df.to_csv('../data/betting.csv', index=False)

## 2. Prepare play-by-play data

In [8]:
# read play-by-play data
pbp_df = pd.concat([
    pd.read_csv("../data/pbp_20.csv"),
    pd.read_csv("../data/pbp_22.csv"),
    pd.read_csv("../data/pbp_23.csv")
], axis=0)

In [9]:
# 0 : game end *
# 1 : made shot *
# 2 : missed shot *
# 3 : free throw *
# 4 : rebound *
# 5 : turnover *
# 6 : foul *
# 7 : violation
# 8 : substitution
# 9 : time out
# 10 : jump ball *
# 11 : ejection
# 12 : start period *
# 13 : end period *
# 20 : stoppage 
# 99 : last possession of quarter *

In [10]:
relevant_events = [0, 1, 2, 3, 4, 5, 6, 10, 11, 12, 13, 99]
pbp_df = pbp_df[pbp_df['msg_type'].isin(relevant_events)]

In [11]:
P_DEF_REB = 0.7
P_DEF_REB_FT = 0.85
P_FTM = 0.7

In [12]:
def identify_event(msg_type, off_team, home_team, description):
    is_in_possession = (home_team == off_team)
    is_actor = (home_team in description)
    if msg_type == 3: #free throw
        if 'flagrant' in description.lower() or 'technical' in description.lower():
            return 33 # technical/flagrant free throw
        if any([f'{x} of {x}' in description for x in (1, 2, 3)]):
            if 'Missed' in description:
                return 31 # last free throw (missed)
            else:
                return 32 # last free throw (made)
    if msg_type == 4: #rebound
        if (not is_in_possession and is_actor) or (is_in_possession and not is_actor):
            return 41 #defensive rebound
        if (not is_in_possession and not is_actor) or (is_in_possession and is_actor):
            return 42 #offensive rebound
    if msg_type == 6: #foul
        is_flagrant = 'flagrant' in description.lower()
        is_technical = 'technical' in description.lower()
        if 'FTA' in description:
            fta = int(description[description.find('FTA')-2])
            if is_flagrant:
                if is_actor == is_in_possession:
                    return 65 #flagrant foul in offense(2 FTA)
                elif is_actor != is_in_possession:
                    return 66 #flagrant foul in defense (2 FTA)
            elif is_technical:
                return 67 #technical foul (1 FTA)
            else:
                if fta == 1:
                    return 61 #personal foul (1 FTA)
                if fta == 2:
                    return 62 # personal foul (2 FTA)
                if fta == 3:
                    return 63 #personal foul (3 FTA)
        if 'offensive' in description.lower():
            return 64 #offensive foul

In [13]:
custom_events = pbp_df.apply(
    lambda x: identify_event(
        x['msg_type'], 
        x['off_slug_team'], 
        x['team_home'], 
        x['description']), 
    axis=1)

In [14]:
pbp_df['event'] = [
    msg_type if np.isnan(custom_event) else custom_event for msg_type, custom_event in zip(pbp_df['msg_type'], custom_events)
]

pbp_df['event'] = pbp_df['event'].astype(int)

In [15]:
event_dict = {
    0: 'game end',
    1: 'field goal (made)',
    2: 'field goal (missed)',
    3: 'free throw',
    31: 'last free throw (missed)',
    32: 'last free throw (made)',
    33: 'technical/flagrant free throw',
    41: 'defensive rebound',
    42: 'offensive rebound',
    5: 'turnover',
    6: 'foul',
    61: 'foul (1 FTA)',
    62: 'foul (2 FTA)',
    63: 'foul (3 FTA)',
    64: 'offensive foul',
    65: 'flagrant foul in offense',
    66: 'flagrant foul in defense',
    67: 'technical foul',
    10: 'jump ball',
    11: 'rejection',
    12: 'start period',
    13: 'end period',
    99: 'last posession of quarter',
}

In [16]:
pbp_df['event_desc'] = pbp_df['event'].apply(lambda x: event_dict[x])

In [17]:
def possession_probs(event, off_team, home_team, period):
    home_team_attacks = int(off_team == home_team)
    if event == 12:
        if period == 1 or period >= 5:
            return 0.5
        else:
            return home_team_attacks
    if event in (0, 13, 99):
        return 0.5
    if event in (1, 32, 41, 5, 64, 65):
        return int(not home_team_attacks)
    if event == 2:
        return abs(home_team_attacks - P_DEF_REB)
    if event == 31:
        return abs(home_team_attacks - P_DEF_REB_FT)
    if event in (10, 11, 33, 42, 6, 66, 67):
        return home_team_attacks
    if event in (3, 61, 62, 63):
        return P_FTM * int(not home_team_attacks) + (1-P_FTM) * abs(home_team_attacks - P_DEF_REB_FT)

In [18]:
pbp_df['home_possession_prob'] = pbp_df.apply(lambda x: 
                                              possession_probs(
                                                  x['event'],
                                                  x['off_slug_team'],
                                                  x['team_home'],
                                                  x['period']
                                              ),
                                              axis=1
                                             )

In [19]:
pbp_df['is_home_team_ejection'] = pbp_df.apply(lambda x: (x['msg_type']==11 or (x['msg_type']==6 and '6 PF' in x['description'] and x['team_home'] in x['description'])), axis=1)
pbp_df['is_away_team_ejection'] = pbp_df.apply(lambda x: (x['msg_type']==11 or (x['msg_type']==6 and '6 PF' in x['description'] and x['team_away'] in x['description'])), axis=1)

pbp_df['home_team_ejections'] = pbp_df.groupby(['game_id', 'period'])['is_home_team_ejection'].cumsum()
pbp_df['away_team_ejections'] = pbp_df.groupby(['game_id', 'period'])['is_away_team_ejection'].cumsum()

In [20]:
pbp_df['ejections'] = pbp_df['home_team_ejections'] - pbp_df['away_team_ejections']

In [21]:
pbp_df['is_home_team_foul'] = pbp_df.apply(lambda x: x['msg_type']==6 and x['event']!=65 and x['team_home'] in x['description'], axis=1)
pbp_df['is_away_team_foul'] = pbp_df.apply(lambda x: x['msg_type'] == 6 and x['event']!=65 and x['team_away'] in x['description'], axis=1)

pbp_df['home_team_fouls'] = pbp_df.groupby(['game_id', 'period'])['is_home_team_foul'].cumsum()
pbp_df['away_team_fouls'] = pbp_df.groupby(['game_id', 'period'])['is_away_team_foul'].cumsum()

In [22]:
pbp_df['l2m'] = pbp_df['clock'].apply(lambda x: int(x[:x.find(':')]) * 60 + int(x[x.find(':')+1:x.find(':')+3])) <= 120

In [23]:
pbp_df['is_home_team_l2m_foul'] = pbp_df['is_home_team_foul'] * pbp_df['l2m']
pbp_df['is_away_team_l2m_foul'] = pbp_df['is_away_team_foul'] * pbp_df['l2m']

C:\Users\p.zukiewicz\AppData\Local\Temp\ipykernel_20224\3633416298.py:1: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead.
  pbp_df['is_home_team_l2m_foul'] = pbp_df['is_home_team_foul'] * pbp_df['l2m']
C:\Users\p.zukiewicz\AppData\Local\Temp\ipykernel_20224\3633416298.py:2: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead.
  pbp_df['is_away_team_l2m_foul'] = pbp_df['is_away_team_foul'] * pbp_df['l2m']


In [24]:
pbp_df['home_l2m_fouls'] = pbp_df.groupby(['game_id', 'period'])['is_home_team_l2m_foul'].cumsum()
pbp_df['away_l2m_fouls'] = pbp_df.groupby(['game_id', 'period'])['is_away_team_l2m_foul'].cumsum()

In [25]:
def is_team_in_bonus(fouls, l2m_fouls, period):
    if period <= 4:
        return fouls >= 5 or l2m_fouls >=2
    if period > 4:
        return fouls >= 4 or l2m_fouls >= 1

In [26]:
pbp_df['home_team_bonus'] = pbp_df.apply(lambda x: is_team_in_bonus(x['home_team_fouls'],
                                                                    x['home_l2m_fouls'],
                                                                    x['period']),
                                        axis=1).astype(int)

pbp_df['away_team_bonus'] = pbp_df.apply(lambda x: is_team_in_bonus(x['away_team_fouls'],
                                                                    x['away_l2m_fouls'],
                                                                    x['period']),
                                        axis=1).astype(int)

In [27]:
pbp_df['bonus'] = pbp_df['home_team_bonus'] - pbp_df['away_team_bonus']

In [28]:
def free_throws_left(home_team, msg_type, event, description):
    if msg_type == 6:
        home_team_fouled = - (2 * (home_team in description) - 1)
        try:
            n_free_throws = int(description[description.find('FTA')-2])
        except ValueError:
            return 0
    elif msg_type == 3:
        home_team_fouled = 2 * (home_team in description) - 1
        if 'technical' in description.lower():
            return 0
        else:
            n_free_throws = int(description[description.find(' of ')+4]) - int(description[description.find(' of ')-1])
    else:
        return 0
    return home_team_fouled * n_free_throws
        

In [29]:
pbp_df['home_team_free_throws'] = pbp_df.apply(lambda x: free_throws_left(x['team_home'], x['msg_type'], x['event'], x['description']), axis=1)

In [30]:
total_game_time = 4 * 12 * 60
overtime = 5 * 60

pbp_df['time_left'] = total_game_time - pbp_df['secs_passed_game']
pbp_df.loc[pbp_df['period'] == 5, 'time_left'] += overtime
pbp_df.loc[pbp_df['period'] == 6, 'time_left'] += 2 * overtime
pbp_df.loc[pbp_df['period'] == 7, 'time_left'] += 3 * overtime
pbp_df.loc[pbp_df['period'] == 8, 'time_left'] += 4 * overtime

In [31]:
pbp_df['margin'] = pbp_df['hs'] - pbp_df['vs']

In [32]:
pbp_df = pd.merge(pbp_df, pbp_df.groupby(['game_id'])[['hs', 'vs']].max().reset_index(), on='game_id', suffixes=('', '_final'))
pbp_df['home_team_won'] = (pbp_df['hs_final'] > pbp_df['vs_final']).astype(int)

In [33]:
eps = 1
x = pbp_df['margin'] / (pbp_df['time_left'] + eps)
pbp_df['time_pressure'] = (np.exp(x) - np.exp(-x)) / (np.exp(x) + np.exp(-x))

In [34]:
pbp_df = pbp_df[['game_date', 'game_id', 'team_home', 'team_away', 'period', 'event_desc', 'home_possession_prob', 'home_team_free_throws', 'bonus', 'ejections', 'margin', 'time_left', 'time_pressure', 'home_team_won']]

## 3. Prepare betting data

In [48]:
betting_df = pd.read_csv('../data/betting.csv')

In [49]:
betting_df = betting_df[['season', 'game_date', 'home_team_abbrev', 'visit_team_abbrev', 'line']]
betting_df['date'] = betting_df['game_date'].apply(lambda x: x[:10])

In [50]:
betting_df['season'] = betting_df['season'].astype(int)
betting_df = betting_df[[season in (2019, 2021, 2022) for season in betting_df['season']]]

## 4. Merge both dataframes

In [247]:
df = pd.merge(pbp_df, betting_df, how='left', left_on=['game_date', 'team_home'], right_on=['date', 'home_team_abbrev'])

In [248]:
df = df.dropna()

In [249]:
df = df[
    [
        'game_id', 
        'date', 
        'team_home', 
        'team_away', 
        'period',
        'event_desc',
        'home_possession_prob',
        'home_team_free_throws',
        'bonus',
        'ejections', 
        'time_left', 
        'margin', 
        'time_pressure', 
        'line', 
        'home_team_won',
    ]
]

In [250]:
df.head()

,game_id,date,team_home,team_away,period,event_desc,home_possession_prob,home_team_free_throws,bonus,ejections,time_left,margin,time_pressure,line,home_team_won
0,21900001,2019-10-22,TOR,NOP,1,start period,0.5,0,0,0,2880.0,0,0.000000,-6.5,1
1,21900001,2019-10-22,TOR,NOP,1,jump ball,0.0,0,0,0,2877.0,0,0.000000,-6.5,1
2,21900001,2019-10-22,TOR,NOP,1,field goal (missed),0.7,0,0,0,2868.0,0,0.000000,-6.5,1
3,21900001,2019-10-22,TOR,NOP,1,offensive rebound,0.0,0,0,0,2867.0,0,0.000000,-6.5,1
4,21900001,2019-10-22,TOR,NOP,1,field goal (made),1.0,0,0,0,2867.0,-2,-0.000697,-6.5,1


In [251]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1424826 entries, 0 to 1439446
Data columns (total 15 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   game_id                1424826 non-null  int64  
 1   date                   1424826 non-null  object 
 2   team_home              1424826 non-null  object 
 3   team_away              1424826 non-null  object 
 4   period                 1424826 non-null  int64  
 5   event_desc             1424826 non-null  object 
 6   home_possession_prob   1424826 non-null  float64
 7   home_team_free_throws  1424826 non-null  int64  
 8   bonus                  1424826 non-null  int32  
 9   ejections              1424826 non-null  int64  
 10  time_left              1424826 non-null  float64
 11  margin                 1424826 non-null  int64  
 12  time_pressure          1424826 non-null  float64
 13  line                   1424826 non-null  float64
 14  home_team_won     

In [252]:
df.to_csv('../data/pbp_data.csv', index=False)